In [1]:
import numpy as np

#myqlm imports
from qat.lang.AQASM import Program, H, RX, RY, RZ, Z, CNOT
from qat.lang.AQASM import *
from qat.qpus import get_default_qpu

## Data embedding 

In order to encoding normalized data, $X$, into quantum states, a angle embedding can be implemented in order to create a quantum analogous to a classical input layer. The normalization of $X$ is showed in main.ipynb file. The quantum operator which represents such encoding used in this example is given by

$$U(X) = \bigotimes_{i=0}^{n}RX(x_i)$$

where $n$ is the number of qubits as well as the number of features in the data. Therefore, the quantum hardware resources complexity in terms of qubits is 

$$n = \mathcal{O}(|X|)$$



In [2]:
def data_embedding(x):
    """
    Args
        x: a np.array containing normalized feature vector;
    Outpu
        emb: a quantum circuit that encodes the data;
        
    """

    emb = QRoutine()
    wires = emb.new_wires(len(x))
    with emb.compute():
        for i, wire in enumerate(wires):
            RX(x[i])(wire)
    
    return emb

## Tunable ansatz

The parametrized quantum circuit used in this quantum neural network is a two-local heuristic pattern using a full entangled quantum circuit. Here, the CNOT's set will be the operator $W$. The mathematical representation of a multi-layer of the ansatz is given by

$$P(\vec{\theta}) = \Pi_{j=1}^{L}\bigotimes_{i=0}^{n}RY(\theta_i)\bigotimes_{i=0}^{n}RZ(\theta_i) (W) \bigotimes_{i=0}^{n}RY(\theta_{i+n})\bigotimes_{i=0}^{n}RZ(\theta_{i+n})$$

where $\vec{\theta}$ is the parameter vector analogous to the weights in a classical neural network. Such parameters are trained via classical optimizers.

In [3]:
def ansatz(params, feature_len, num_layers):
    """
    Args
        params: np.array with tunable parameters in the ansatz;
        feature_len: a integer which is the size of feature vector (atributes);
        num_layers: a integer which is the number of layers;
    Outpu
        pcirc: a quantum circuit which is the ansatz;
        
    """  

    pcirc = QRoutine()
    wires = pcirc.new_wires(feature_len)
    #writing quantum circuit for the ansatz
    for layer in range(num_layers):
        with pcirc.compute():
            for i, wire in enumerate(wires):
                RY(params[i + layer*feature_len])(wire)
                RZ(params[i + layer*feature_len])(wire)          
            if layer == num_layers:
                break
            
            #circular entanglement
            CNOT(wires[0], wires[1])
            CNOT(wires[0], wires[2])
            CNOT(wires[0], wires[3])
            CNOT(wires[1], wires[2])
            CNOT(wires[1], wires[3])            
            CNOT(wires[2], wires[3])
            
    return pcirc